## AlphaGenome predictions data wrangling

### AlphaGenome predictions with RNA-seq modality

In [ ]:
### modify this to convert the .tsv from AlphaGenome to .parquet for the model_benchmarking notebook. It was already done and the db directory
### already has the output .parquet file in the required format, but I forgot to save the notebook, so lost the code. 

### To note: I adjusted the format to match the .parquet format needed for the AUC calculation and removed all rows with NAN values 
### The final df has ~17000 rows 
### Also create a version where all the NAN values are converted into zeros

import pandas as pd
import numpy as np

alphagenome = pd.read_csv('/hpc/group/igvf/kk319/repo/Proj_IGVF_BlueSTARR/results/predict_variant_kircher2019/result.alphagenome.variant.summary.tsv', sep='\t')
alphagenome[['chrom','cre_start','cre_end']] = alphagenome['Region'].str.split(r'[:\-]', expand=True)
alphagenome['Scored_Interval'] = alphagenome['Scored_Interval'].str.rstrip(r'[:\.]')
alphagenome['Scored_Interval'] = alphagenome['Scored_Interval'].fillna(alphagenome['Region'])

alphagenome[['bin_chrom','bin_start','bin_end']] = alphagenome['Scored_Interval'].str.split(r'[:\-]', expand=True)
alphagenome['Raw_Score'] = pd.to_numeric(alphagenome['Raw_Score'], errors='coerce')
mask_alt = alphagenome['Ref'] != alphagenome['Alt']
alphagenome['log2FC'] = np.nan
alphagenome.loc[mask_alt, 'log2FC'] = alphagenome.loc[mask_alt, 'Raw_Score']
drop_mask = (alphagenome['Ref'] == alphagenome['Alt']) & (alphagenome['Raw_Score'] == 0)
alphagenome = alphagenome.loc[~drop_mask].copy()
alphagenome.reset_index(drop=True, inplace=True)


alphagenome.rename(columns={'Position':'allele_pos',
                            'Ref':'ref_allele',
                            'Alt':'allele',
                            'Raw_Score':'effect'}, inplace=True)
alphagenome_v1 = alphagenome[['chrom','cre_start','cre_end','bin_chrom','bin_start','bin_end','allele_pos','ref_allele','allele','effect','log2FC']]
alphagenome_v1.to_csv('/hpc/home/rv103/igvf/revathy/Blueplots/predictions/K562_AlphaGenome.txt', sep='\t', index=False)



In [ ]:
min_col = alphagenome_v1['log2FC'].min()
alphagenome_v1['effect'] = alphagenome_v1['effect'].fillna(min_col)
alphagenome_v1['log2FC'] = alphagenome_v1['log2FC'].fillna(min_col)

alphagenome_v1.to_parquet('/hpc/home/rv103/igvf/revathy/db/Kircher-2019_K562-AlphaGenome.parquet', engine='pyarrow', compression='snappy')

### AlphaGenome with ATAC-seq modality

In [ ]:
import pandas as pd
import numpy as np

alphagenome = pd.read_csv('/hpc/group/igvf/kk319/repo/Proj_IGVF_BlueSTARR/results/predict_variant_kircher2019/result.alphagenome.K562.ATAC.2KB.variant.summary.tsv', sep='\t')
alphagenome[['chrom','cre_start','cre_end']] = alphagenome['Region'].str.split(r'[:\-]', expand=True)
alphagenome['Scored_Interval'] = alphagenome['Scored_Interval'].str.rstrip(r'[:\.]')
alphagenome['Scored_Interval'] = alphagenome['Scored_Interval'].fillna(alphagenome['Region'])

alphagenome[['bin_chrom','bin_start','bin_end']] = alphagenome['Scored_Interval'].str.split(r'[:\-]', expand=True)
alphagenome['Raw_Score'] = pd.to_numeric(alphagenome['Raw_Score'], errors='coerce')
mask_alt = alphagenome['Ref'] != alphagenome['Alt']
alphagenome['log2FC'] = np.nan
alphagenome.loc[mask_alt, 'log2FC'] = alphagenome.loc[mask_alt, 'Raw_Score']
drop_mask = (alphagenome['Ref'] == alphagenome['Alt']) & (alphagenome['Raw_Score'] == 0)
alphagenome = alphagenome.loc[~drop_mask].copy()
alphagenome.reset_index(drop=True, inplace=True)


alphagenome.rename(columns={'Position':'allele_pos',
                            'Ref':'ref_allele',
                            'Alt':'allele',
                            'Raw_Score':'effect'}, inplace=True)
alphagenome_v1 = alphagenome[['chrom','cre_start','cre_end','bin_chrom','bin_start','bin_end','allele_pos','ref_allele','allele','effect','log2FC']]
alphagenome_v1.to_csv('/hpc/home/rv103/igvf/revathy/Blueplots/predictions/K562_AlphaGenome-ATACseq.txt', sep='\t', index=False)

# min_col = alphagenome_v1['log2FC'].min()
# alphagenome_v1['effect'] = alphagenome_v1['effect'].fillna(min_col)
# alphagenome_v1['log2FC'] = alphagenome_v1['log2FC'].fillna(min_col)

alphagenome_v1.to_parquet('/hpc/home/rv103/igvf/revathy/db/Kircher-2019_K562-AlphaGenome-ATACseq.parquet', engine='pyarrow', compression='snappy')


